In [22]:
import pandas as pd
import networkx as nx
import numpy as np

# Read in the CSV files

# 55% SAR count
#file_directory = "../datasets/60K_60_sar_count/"

# 10% SAR count
#file_directory = "../datasets/60K_10_sar_count/"

# 5% SAR count
#file_directory = "../datasets/60K_05_sar_count/"

# 2% SAR count
file_directory = "../datasets/60K_01_sar_count/"

accounts_df = pd.read_csv(file_directory+"account_attributes.csv")
transactions_df = pd.read_csv(file_directory+"transactions.csv")


nodes_df = accounts_df
edges_df = transactions_df

import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.transforms import NormalizeFeatures

x_np = nodes_df.to_numpy()
x = x_np[:,0:-2]
print(x.shape, x_np.shape)

from torch.utils.data import random_split
# Define your graph
x = torch.nn.functional.normalize(torch.tensor(x),dim=0).to(torch.float32)  # (n x features)
edge_index =  torch.stack([torch.tensor(edges_df.orig_acct.to_numpy()),torch.tensor(edges_df.bene_acct.to_numpy())],dim=-1).T  # Define your edge index
#edge_attr = torch.nn.functional.normalize(torch.tensor(np.array(edges_df[['amount','oldBalanceOrig', 'newBalanceOrig', 'oldBalanceDest', 'newBalanceDest','isFlaggedFraud','isUnauthorizedOverdraft','action_CASH_IN','action_CASH_OUT','action_DEBIT','action_PAYMENT','action_TRANSFER']].values,dtype='float32')),dim=0) # edge features
edge_weight = torch.nn.functional.normalize(torch.tensor(edges_df.base_amt.to_numpy()),dim=0).long()
y =  torch.tensor(nodes_df.node_isSar.to_numpy().astype(int),dtype=torch.long) # target values

# Bitflip the tensor
#y = y ^ torch.tensor([1], dtype=torch.uint8).expand_as(y)


train_size = int(0.35 * len(y))  # 60% of the dataset for training
val_size = int(0.15 * len(y))    # 20% of the dataset for validation
test_size = len(y) - train_size - val_size  # Remaining 20% for testing

train_dataset, val_dataset, test_dataset = random_split(y, [train_size, val_size, test_size])

# Create masks for train, validation, and test sets
train_mask = torch.zeros(len(y), dtype=torch.bool)
val_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)

train_mask[train_dataset.indices] = True
val_mask[val_dataset.indices] = True
test_mask[test_dataset.indices] = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from torch_geometric.data import Data
# Load your data into PyTorch Geometric's Data class
#data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y,train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
#data = Data(x=x, edge_index=edge_index, y=y,train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y,train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
data.to(device)

from models.MPNN import GNN_MPNN_Model
from models.SAGE import GNN_SAGE_Model
from models.CONV import GNN_CONV_Model
from torch_geometric.nn import Sequential, GAT, GIN, GraphSAGE
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support
import json
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))


2.0.1+cu118
(60215, 21) (60215, 23)
cuda:0


<IPython.core.display.Javascript object>

In [2]:
model_MPNN = torch.load(file_directory + 'trained_models/MPNN_best_model.pth')
model_SAGE = torch.load(file_directory + 'trained_models/SAGE_best_model.pth')
model_GCN = torch.load(file_directory + 'trained_models/GCN_best_model.pth')
model_GAT = torch.load(file_directory + 'trained_models/GAT_best_model.pth')
model_GIN = torch.load(file_directory + 'trained_models/GIN_best_model.pth')

### since test and training sets are different now to when we originally trained the models, we need to retrain using the best parameters before doing statistics

In [23]:
#load best models json

f = open(file_directory + 'trained_models/MPNN_best_model.json')
MPNN_JSON = json.load(f)

f = open(file_directory + 'trained_models/SAGE_best_model.json')
SAGE_JSON = json.load(f)

f = open(file_directory + 'trained_models/GCN_best_model.json')
GCN_JSON = json.load(f)

f = open(file_directory + 'trained_models/GAT_best_model.json')
GAT_JSON = json.load(f)

f = open(file_directory + 'trained_models/GIN_best_model.json')
GIN_JSON = json.load(f)

In [24]:
from models.ModelGS import ModelGS
dataset_num_features = x.size()[1]
dataset_num_classes = 2

In [25]:
#MPNN
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.15,1.0]).to(device))
model = GNN_MPNN_Model(hidden_size=MPNN_JSON['layer size'], input_size=dataset_num_features, output_size=dataset_num_classes, num_layers=MPNN_JSON['num layers']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=MPNN_JSON['learning rate'], weight_decay=5e-4)
epochs = 1000
model_MPNN2 = ModelGS(model, optimizer, criterion, data)
print("training MPNN")
for epoch in range(0, epochs):
        train_loss = model_MPNN2.train()
        val_loss, val_out, val_pred = model_MPNN2.test(data.val_mask)

#SAGE
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.15,1.0]).to(device))
model = GNN_SAGE_Model(hidden_size=MPNN_JSON['layer size'], input_size=dataset_num_features, output_size=dataset_num_classes, num_layers=MPNN_JSON['num layers']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=MPNN_JSON['learning rate'], weight_decay=5e-4)
epochs = 1000
model_SAGE2 = ModelGS(model, optimizer, criterion, data)
print("training SAGE")
for epoch in range(0, epochs):
        train_loss = model_SAGE2.train()
        val_loss, val_out, val_pred = model_SAGE2.test(data.val_mask)


#GCN
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.15,1.0]).to(device))
model = GNN_CONV_Model(hidden_size=MPNN_JSON['layer size'], input_size=dataset_num_features, output_size=dataset_num_classes, num_layers=MPNN_JSON['num layers']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=MPNN_JSON['learning rate'], weight_decay=5e-4)
epochs = 1000
model_CONV2 = ModelGS(model, optimizer, criterion, data)
print("training GCN")
for epoch in range(0, epochs):
        train_loss = model_CONV2.train()
        val_loss, val_out, val_pred = model_CONV2.test(data.val_mask)


#GAT
criterion = torch.nn.CrossEntropyLoss()
model = GAT(in_channels=dataset_num_features, hidden_channels=MPNN_JSON['layer size'], num_layers=MPNN_JSON['num layers'], out_channels=dataset_num_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=MPNN_JSON['learning rate'], weight_decay=5e-4)
epochs = 1000
model_GAT2 = ModelGS(model, optimizer, criterion, data)
print("training GAT")
for epoch in range(0, epochs):
        train_loss = model_GAT2.train()
        val_loss, val_out, val_pred = model_GAT2.test(data.val_mask)


#GIN
criterion = torch.nn.CrossEntropyLoss()
model = GIN(in_channels=dataset_num_features, hidden_channels=MPNN_JSON['layer size'], num_layers=MPNN_JSON['num layers'], out_channels=dataset_num_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=MPNN_JSON['learning rate'], weight_decay=5e-4)
epochs = 2000
model_GIN2 = ModelGS(model, optimizer, criterion, data)
print("training GIN")
for epoch in range(0, epochs):
        train_loss = model_GIN2.train()
        val_loss, val_out, val_pred = model_GIN2.test(data.val_mask)

print("Done")


training MPNN
training SGE
training GCN
training GAT
training GIN
Done


In [26]:
sar_df = pd.read_csv(file_directory+"sar_accounts.csv")
ids_cycle = sar_df["ACCOUNT_ID"][sar_df["ALERT_TYPE"]=='cycle'].to_numpy() #super important to convert to numpy array!!!!! keeping as pandas series messes up all indexing!!!
ids_fan_in = sar_df["ACCOUNT_ID"][sar_df["ALERT_TYPE"]=='fan_in'].to_numpy()
ids_fan_out = sar_df["ACCOUNT_ID"][sar_df["ALERT_TYPE"]=='fan_out'].to_numpy()
ids_gather_scatter = sar_df["ACCOUNT_ID"][sar_df["ALERT_TYPE"]=='gather_scatter'].to_numpy()
ids_scatter_gather = sar_df["ACCOUNT_ID"][sar_df["ALERT_TYPE"]=='scatter_gather'].to_numpy()
set(sar_df["ALERT_TYPE"])

ids_test_set = accounts_df['id'][data.test_mask.cpu().numpy()].astype(int).to_numpy()
labels_test_set = data.y[data.test_mask].cpu().numpy()

_, _, test_pred  = model_MPNN2.test(data.test_mask)
predictions_test_set_MPNN = test_pred.cpu().numpy()

_, _, test_pred  = model_SAGE2.test(data.test_mask)
predictions_test_set_SAGE = test_pred.cpu().numpy()

_, _, test_pred  = model_CONV2.test(data.test_mask)
predictions_test_set_GCN = test_pred.cpu().numpy()

_, _, test_pred  = model_GAT2.test(data.test_mask)
predictions_test_set_GAT = test_pred.cpu().numpy()

_, _, test_pred  = model_GIN2.test(data.test_mask)
predictions_test_set_GIN = test_pred.cpu().numpy()

correct_pos_pred_MPNN = np.logical_and(labels_test_set, predictions_test_set_MPNN) # is can only equal 1 if both label and precition were 1
correct_pos_pred_SAGE = np.logical_and(labels_test_set, predictions_test_set_SAGE)
correct_pos_pred_GCN = np.logical_and(labels_test_set, predictions_test_set_GCN)
correct_pos_pred_GAT = np.logical_and(labels_test_set, predictions_test_set_GAT)
correct_pos_pred_GIN = np.logical_and(labels_test_set, predictions_test_set_GIN)


pred_cycle_MPNN = correct_pos_pred_MPNN[[e in ids_cycle for e in ids_test_set]]
pred_fan_in_MPNN = correct_pos_pred_MPNN[[e in ids_fan_in for e in ids_test_set]]
pred_fan_out_MPNN = correct_pos_pred_MPNN[[e in ids_fan_out for e in ids_test_set]]
pred_gather_scatter_MPNN = correct_pos_pred_MPNN[[e in ids_gather_scatter for e in ids_test_set]]
pred_scatter_gather_MPNN = correct_pos_pred_MPNN[[e in ids_scatter_gather for e in ids_test_set]]


pred_cycle_SAGE = correct_pos_pred_SAGE[[e in ids_cycle for e in ids_test_set]]
pred_fan_in_SAGE = correct_pos_pred_SAGE[[e in ids_fan_in for e in ids_test_set]]
pred_fan_out_SAGE = correct_pos_pred_SAGE[[e in ids_fan_out for e in ids_test_set]]
pred_gather_scatter_SAGE = correct_pos_pred_SAGE[[e in ids_gather_scatter for e in ids_test_set]]
pred_scatter_gather_SAGE = correct_pos_pred_SAGE[[e in ids_scatter_gather for e in ids_test_set]]


pred_cycle_GCN = correct_pos_pred_GCN[[e in ids_cycle for e in ids_test_set]]
pred_fan_in_GCN = correct_pos_pred_GCN[[e in ids_fan_in for e in ids_test_set]]
pred_fan_out_GCN = correct_pos_pred_GCN[[e in ids_fan_out for e in ids_test_set]]
pred_gather_scatter_GCN = correct_pos_pred_GCN[[e in ids_gather_scatter for e in ids_test_set]]
pred_scatter_gather_GCN = correct_pos_pred_GCN[[e in ids_scatter_gather for e in ids_test_set]]


pred_cycle_GAT = correct_pos_pred_GAT[[e in ids_cycle for e in ids_test_set]]
pred_fan_in_GAT = correct_pos_pred_GAT[[e in ids_fan_in for e in ids_test_set]]
pred_fan_out_GAT = correct_pos_pred_GAT[[e in ids_fan_out for e in ids_test_set]]
pred_gather_scatter_GAT = correct_pos_pred_GAT[[e in ids_gather_scatter for e in ids_test_set]]
pred_scatter_gather_GAT = correct_pos_pred_GAT[[e in ids_scatter_gather for e in ids_test_set]]


pred_cycle_GIN = correct_pos_pred_GIN[[e in ids_cycle for e in ids_test_set]]
pred_fan_in_GIN = correct_pos_pred_GIN[[e in ids_fan_in for e in ids_test_set]]
pred_fan_out_GIN = correct_pos_pred_GIN[[e in ids_fan_out for e in ids_test_set]]
pred_gather_scatter_GIN = correct_pos_pred_GIN[[e in ids_gather_scatter for e in ids_test_set]]
pred_scatter_gather_GIN = correct_pos_pred_GIN[[e in ids_scatter_gather for e in ids_test_set]]

In [27]:
topologies = ('Cycle', 'Fan In', 'Fan Out', 'Gather Scatter', 'Scatter Gather')

tp_MPNN = np.round(np.array([np.sum(pred_cycle_MPNN)/len(pred_cycle_MPNN), np.sum(pred_fan_in_MPNN)/len(pred_fan_in_MPNN), np.sum(pred_fan_out_MPNN)/len(pred_fan_out_MPNN), np.sum(pred_gather_scatter_MPNN)/len(pred_gather_scatter_MPNN), np.sum(pred_scatter_gather_MPNN)/len(pred_scatter_gather_MPNN)]),2)

tp_SAGE = np.round(np.array([np.sum(pred_cycle_SAGE)/len(pred_cycle_SAGE), np.sum(pred_fan_in_SAGE)/len(pred_fan_in_SAGE), np.sum(pred_fan_out_SAGE)/len(pred_fan_out_SAGE), np.sum(pred_gather_scatter_SAGE)/len(pred_gather_scatter_SAGE), np.sum(pred_scatter_gather_SAGE)/len(pred_scatter_gather_SAGE)]),2)

tp_GCN = np.round(np.array([np.sum(pred_cycle_GCN)/len(pred_cycle_GCN), np.sum(pred_fan_in_GCN)/len(pred_fan_in_GCN), np.sum(pred_fan_out_GCN)/len(pred_fan_out_GCN), np.sum(pred_gather_scatter_GCN)/len(pred_gather_scatter_GCN), np.sum(pred_scatter_gather_GCN)/len(pred_scatter_gather_GCN)]),2)

tp_GAT = np.round(np.array([np.sum(pred_cycle_GAT)/len(pred_cycle_GAT), np.sum(pred_fan_in_GAT)/len(pred_fan_in_GAT), np.sum(pred_fan_out_GAT)/len(pred_fan_out_GAT), np.sum(pred_gather_scatter_GAT)/len(pred_gather_scatter_GAT), np.sum(pred_scatter_gather_GAT)/len(pred_scatter_gather_GAT)]),2)

tp_GIN = np.round(np.array([np.sum(pred_cycle_GIN)/len(pred_cycle_GIN), np.sum(pred_fan_in_GIN)/len(pred_fan_in_GIN), np.sum(pred_fan_out_GIN)/len(pred_fan_out_GIN), np.sum(pred_gather_scatter_GIN)/len(pred_gather_scatter_GIN), np.sum(pred_scatter_gather_GIN)/len(pred_scatter_gather_GIN)]),2)


fp_MPNN = np.ones(len(tp_MPNN)) - tp_MPNN
fp_SAGE = np.ones(len(tp_SAGE)) - tp_SAGE
fp_GCN = np.ones(len(tp_GCN)) - tp_GCN
fp_GAT = np.ones(len(tp_GAT)) - tp_GAT
fp_GIN = np.ones(len(tp_GIN)) - tp_GIN

print("MPNN:",tp_MPNN, fp_MPNN)
print("SAGE:",tp_SAGE, fp_SAGE)
print("GCN:",tp_GCN, fp_GCN)
print("GAT:",tp_GAT, fp_GAT)
print("GIT:",tp_GIN, fp_GIN)

'''
topology_counts = {
    'True Positive': tp,
    'False Positive': fp,
}
width = 0.6  # the width of the bars: can also be len(x) sequence


fig, ax = plt.subplots()
bottom = np.zeros(5)

for top, top_count in topology_counts.items():
    p = ax.bar(topologies, top_count, width, label=top, bottom=bottom)
    bottom += top_count

    ax.bar_label(p, label_type='center')

ax.set_title('Model Performance by Topology Type')
ax.legend()

plt.show()

'''

MPNN: [0.78 0.73 0.19 0.85 0.88] [0.22 0.27 0.81 0.15 0.12]
SAGE: [0.84 0.83 0.37 0.87 0.91] [0.16 0.17 0.63 0.13 0.09]
GCN: [0. 0. 0. 0. 0.] [1. 1. 1. 1. 1.]
GAT: [0.56 0.62 0.   0.85 0.71] [0.44 0.38 1.   0.15 0.29]
GIT: [0. 0. 0. 0. 0.] [1. 1. 1. 1. 1.]


"\ntopology_counts = {\n    'True Positive': tp,\n    'False Positive': fp,\n}\nwidth = 0.6  # the width of the bars: can also be len(x) sequence\n\n\nfig, ax = plt.subplots()\nbottom = np.zeros(5)\n\nfor top, top_count in topology_counts.items():\n    p = ax.bar(topologies, top_count, width, label=top, bottom=bottom)\n    bottom += top_count\n\n    ax.bar_label(p, label_type='center')\n\nax.set_title('Model Performance by Topology Type')\nax.legend()\n\nplt.show()\n\n"

### Best models table generation helper

In [28]:
# 55% SAR count
file_directory = "../datasets/60K_60_sar_count/"

f = open(file_directory + 'trained_models/MPNN_best_model.json')
MPNN_JSON_D1 = json.load(f)

f = open(file_directory + 'trained_models/SAGE_best_model.json')
SAGE_JSON_D1 = json.load(f)

f = open(file_directory + 'trained_models/GCN_best_model.json')
GCN_JSON_D1 = json.load(f)

f = open(file_directory + 'trained_models/GAT_best_model.json')
GAT_JSON_D1 = json.load(f)

f = open(file_directory + 'trained_models/GIN_best_model.json')
GIN_JSON_D1 = json.load(f)

# 10% SAR count
file_directory = "../datasets/60K_10_sar_count/"

f = open(file_directory + 'trained_models/MPNN_best_model.json')
MPNN_JSON_D2 = json.load(f)

f = open(file_directory + 'trained_models/SAGE_best_model.json')
SAGE_JSON_D2 = json.load(f)

f = open(file_directory + 'trained_models/GCN_best_model.json')
GCN_JSON_D2 = json.load(f)

f = open(file_directory + 'trained_models/GAT_best_model.json')
GAT_JSON_D2 = json.load(f)

f = open(file_directory + 'trained_models/GIN_best_model.json')
GIN_JSON_D2 = json.load(f)

# 5% SAR count
file_directory = "../datasets/60K_05_sar_count/"

f = open(file_directory + 'trained_models/MPNN_best_model.json')
MPNN_JSON_D3 = json.load(f)

f = open(file_directory + 'trained_models/SAGE_best_model.json')
SAGE_JSON_D3 = json.load(f)

f = open(file_directory + 'trained_models/GCN_best_model.json')
GCN_JSON_D3 = json.load(f)

f = open(file_directory + 'trained_models/GAT_best_model.json')
GAT_JSON_D3 = json.load(f)

f = open(file_directory + 'trained_models/GIN_best_model.json')
GIN_JSON_D3 = json.load(f)

# 2% SAR count
file_directory = "../datasets/60K_01_sar_count/"

f = open(file_directory + 'trained_models/MPNN_best_model.json')
MPNN_JSON_D4 = json.load(f)

f = open(file_directory + 'trained_models/SAGE_best_model.json')
SAGE_JSON_D4 = json.load(f)

f = open(file_directory + 'trained_models/GCN_best_model.json')
GCN_JSON_D4 = json.load(f)

f = open(file_directory + 'trained_models/GAT_best_model.json')
GAT_JSON_D4 = json.load(f)

f = open(file_directory + 'trained_models/GIN_best_model.json')
GIN_JSON_D4 = json.load(f)

In [ ]:
print('')

### Dataset sizes for plot

In [5]:
import pandas as pd
import numpy as np

# 55% SAR count
file_directory = "../datasets/60K_60_sar_count/"
accounts_df_55 = pd.read_csv(file_directory+"account_attributes.csv")
transactions_df_55 = pd.read_csv(file_directory+"transactions.csv")

# 10% SAR count
file_directory = "../datasets/60K_10_sar_count/"
accounts_df_10 = pd.read_csv(file_directory+"account_attributes.csv")
transactions_df_10 = pd.read_csv(file_directory+"transactions.csv")

# 5% SAR count
#file_directory = "../datasets/60K_05_sar_count/"
accounts_df_5 = pd.read_csv(file_directory+"account_attributes.csv")
transactions_df_5 = pd.read_csv(file_directory+"transactions.csv")

# 2% SAR count
#file_directory = "../datasets/60K_01_sar_count/"
accounts_df_2 = pd.read_csv(file_directory+"account_attributes.csv")
transactions_df_2 = pd.read_csv(file_directory+"transactions.csv")

In [14]:
print("Dataset 1")
print("Number of vertices:",len(accounts_df_55))
print("Number of anomalous:",len(accounts_df_55[accounts_df_55["node_isSar"] == 1]))
print("Number of normal:",len(accounts_df_55[accounts_df_55["node_isSar"] == 0]))
print("Number of edges:", len(transactions_df_55))
print()

print("Dataset 2")
print("Number of vertices:",len(accounts_df_10))
print("Number of anomalous:",len(accounts_df_10[accounts_df_10["node_isSar"] == 1]))
print("Number of normal:",len(accounts_df_10[accounts_df_10["node_isSar"] == 0]))
print("Number of edges:", len(transactions_df_10))
print()

print("Dataset 3")
print("Number of vertices:",len(accounts_df_5))
print("Number of anomalous:",len(accounts_df_5[accounts_df_5["node_isSar"] == 1]))
print("Number of normal:",len(accounts_df_5[accounts_df_5["node_isSar"] == 0]))
print("Number of edges:", len(transactions_df_5))
print()

print("Dataset 4")
print("Number of vertices:",len(accounts_df_2))
print("Number of anomalous:",len(accounts_df_2[accounts_df_2["node_isSar"] == 1]))
print("Number of normal:",len(accounts_df_2[accounts_df_2["node_isSar"] == 0]))
print("Number of edges:", len(transactions_df_2))
print()

Dataset 1
Number of vertices: 60215
Number of anomalous: 32877
Number of normal: 27338
Number of edges: 1076063

Dataset 2
Number of vertices: 60215
Number of anomalous: 6581
Number of normal: 53634
Number of edges: 1001080

Dataset 3
Number of vertices: 60215
Number of anomalous: 6581
Number of normal: 53634
Number of edges: 1001080

Dataset 4
Number of vertices: 60215
Number of anomalous: 6581
Number of normal: 53634
Number of edges: 1001080

